In [2]:
import selenium
from selenium import webdriver
import time
import requests
from PIL import Image
import io
from io import BytesIO
import os
import hashlib


In [3]:
DRIVER_PATH = "/Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/chromedriver"
wd = webdriver.Chrome(executable_path= DRIVER_PATH)

/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_34071/708202962.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path= DRIVER_PATH)


In [4]:
wd.get('https://google.com')



In [5]:
# search_box = wd.find_element_by_css_selector('input.gLFyf')
# search_box.send_keys('Dogs')

In [6]:
wd.quit()

In [7]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return 
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [8]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

Persist all images

In [9]:

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

Searching download and saving images

In [10]:
search_term_cutlery = "plastic cutlery"
search_term_cups = "plastic cup"
search_term_bottle = "plastic bottle"
search_term_cutlery_dirty = "dirty plastic cutlery"
search_term_cups_dirty = " dirty plastic cup"
search_term_bottle_dirty = " dirty plastic bottle"
driver_path = DRIVER_PATH
target_path = "/Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images"
number_images = 10
search_and_download(search_term_bottle, driver_path, target_path, number_images)
search_and_download(search_term_cups, driver_path, target_path, number_images)
search_and_download(search_term_cutlery, driver_path, target_path, number_images)

search_and_download(search_term_bottle_dirty, driver_path, target_path, number_images)
search_and_download(search_term_cups_dirty, driver_path, target_path, number_images)
search_and_download(search_term_cutlery_dirty, driver_path, target_path, number_images)

/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_34071/2975638876.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_34071/2109922686.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


/var/folders/q8/n1xznm2s4k92f_f_2r5cxygc0000gn/T/ipykernel_34071/2109922686.py:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 10 image links, done!
SUCCESS - saved https://img.myloview.fr/posters/vector-flat-plastic-bottle-icon-400-193694578.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/29b851a497.jpg
SUCCESS - saved https://media.istockphoto.com/vectors/plastic-bottle-with-mineral-water-on-alpha-transparent-background-vector-id886269012 - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/a3d5417475.jpg
SUCCESS - saved https://media.istockphoto.com/photos/water-plastic-bottle-isolated-picture-id926200658 - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/2c679cef83.jpg
SUCCESS - saved https://static.turbosquid.com/Preview/001236/383/UA/plastic-bottle-3D-model_600.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/285cf1d3ea.jpg
SUCCESS - saved https://m.media-amazon.com/images/I/61Krpl7FGyL._SL1500_.jp

/Users/enyonadjanor/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://images.twinkl.co.uk/tr/image/upload/t_illustration/illustation/plastic-bottle.png - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/9f4c180665.jpg
SUCCESS - saved https://static.turbosquid.com/Preview/001235/567/KI/plastic-water-bottle-3D-model_600.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/a70afd01a7.jpg
SUCCESS - saved https://souriredessaveurs.com/3745-large_default/source-water-pet-plastic-bottle-50-cl.jpg - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/f04e5be4eb.jpg
SUCCESS - saved https://media.istockphoto.com/photos/plastic-bottle-group-on-white-picture-id906026600 - as /Users/enyonadjanor/IA-P2-Euskadi-Enyon/Projets/Projet P8 - Triof/triof/images/plastic_bottle/995c6868fa.jpg
SUCCESS - saved https://media.istockphoto.com/photos/plastic-bottles-picture-id466252351?k=20&m=466252351&s=170667a&w=0&h=